<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-30 12:21:44
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.35 C
-------------------
Today PnL: 52.40 K (0.39%)
Current PnL: -14.74 L (-10.27%)
CY Booked + Current PnL: -8.00 L (-5.58%)
-------------------
Total profit:  3.77 L
Total loss:  -18.52 L
-------------------
Total Booked + Current PnL: 18.98 L (16.28%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (5.0%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.07 C
Est FTT PnL: 72.06 L (53.46%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 9.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
84  UJJIVANSFB    52.77    53.00    54.03       M-SC  94.62  133407.0   
3   ADANIPORTS  1103.69  1583.00     2.03       M-LC  44.09  102595.0   
36    HINDALCO   651.95   761.55   -19.08       H-LC  87.10  111560.0   
68        SBIN   760.30   863.00   -10.56       M-LC  39.78  217035.0   
78   TATAELXSI  7332.28  7332.00   -15.80       X-MC  24.73   88326.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
84      -9072.0   9699.0        1.27         -6.37   7.27   0.44    243.0   
3       24233.0   9798.0        0.33         30.92   9.55  43.43     77.0   
36       7248.0  10286.0       -0.01          6.95   9.22  16.81     27.0   
68      15556.0  11655.0        1.69          7.72   5.37  13.51     76.0   
78     -14326.0  14318.0       -0.57        -13.96  16.21  -0.00     58.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
84    -0.94        0.99    57.46    OX40N      NTT    BANKS  
3      2.47        0.76    37.89     XY24      NTT     MISC  
36     0.70        0.83    24.11      X5K      ATH   METALS  
68     1.33        1.61    21.29     XY25      NTT    BANKS  
78    -1.00        0.66    34.27    OX40N      NTT       IT

In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
52  LALPATHLAB  2804.19  3545.00   -33.15       H-SC  36.56  153725.0   
15  BLUESTARCO  1646.70  2337.55   -13.09       H-SC  74.19  163050.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
52       -505.0  41244.0        0.94         -0.33  26.83  26.42    125.0   
15      -1620.0  70698.0        0.40         -0.98  43.36  41.95    121.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
52    -0.01        1.14    21.23     X40N      NTT  HEALTHCARE  
15    -0.02        1.21     6.41     X40N      ATH          AC

In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
67       SATIN   199.77   274.00   -18.53       H-SC   71.51  161973.0   
4   ALKYLAMINE  2347.98  4546.37     8.10       X-SC  100.00  100564.0   
51    KPIGREEN   497.21   732.16    25.90       H-SC   62.37  128810.0   
11  BANDHANBNK   214.25   400.00   -25.73       H-SC   89.25  221096.0   
64       ROUTE  1558.40  2249.26   -36.92       M-SC   78.49   67650.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
67     -34001.0  106821.0        4.97        -17.35   65.95  37.16    130.0   
4        -399.0   94932.0        3.40         -0.40   94.40  93.63     64.0   
51       3513.0   55697.0        3.24          2.80   43.24  47.25    139.0   
11     -27434.0  242896.0        2.69        -11.04  109.86  86.70    129.0   
64     -35204.0   80801.0        2.57        -34.23  119.44  44.33    192.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
67    -0.32        1.20    20.96     XY24      NTT    FINANCE  
4     -0.00        0.75    49.79       SR      ATH  CHEMICALS  
51     0.06        0.96    61.15       MH      ATH      POWER  
11    -0.11        1.64    46.27     XY24      NTT      BANKS  
64    -0.44        0.50    12.34       SR      ATH         IT

In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
42   INDIAMART  2327.09  4871.06   -47.89       H-SC  88.17  135664.0   
33     HAVELLS  1588.02  2069.16    -7.02       X-MC  41.94  240349.0   
35  HEROMOTOCO  4311.81  6039.03    -7.14       H-MC  20.43  149096.0   
89      VIPIND   488.80   489.00  -211.21       H-SC  91.40   80223.0   
5   APOLLOHOSP  6904.43  8285.00   -20.50       H-LC  64.52  108330.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
42      12328.0  122505.0       -1.86         10.00  90.30  109.32    122.0   
33      -7382.0   82440.0       -1.67         -2.98  34.30   30.30     53.0   
35      -1817.0   62262.0       -1.40         -1.20  41.76   40.06    102.0   
89     -15093.0   15130.0       -1.39        -15.83  18.86    0.04    154.0   
5        4764.0   15946.0       -1.18          4.60  14.72   20.00     30.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
42     0.10        1.01    34.03       AR      ATH        MISC  
33    -0.09        1.78     9.92      X40      ATH  ELECTRICAL  
35    -0.03        1.11    21.95       AR      ATH        AUTO  
89    -1.00        0.60    58.23    OX40N      NTT        MISC  
5      0.30        0.80    19.32     X40N      BTT  HEALTHCARE

In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
84  UJJIVANSFB    52.77    53.0    54.03       M-SC  94.62  133407.0   
78   TATAELXSI  7332.28  7332.0   -15.80       X-MC  24.73   88326.0   
89      VIPIND   488.80   489.0  -211.21       H-SC  91.40   80223.0   
43  INDIGOPNTS  1407.73  1408.0   122.66       H-SC  84.95  145824.0   
50   KANSAINER   299.63   340.0   -67.65       H-SC  22.04  224685.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
84      -9072.0   9699.0        1.27         -6.37   7.27   0.44    243.0   
78     -14326.0  14318.0       -0.57        -13.96  16.21  -0.00     58.0   
89     -15093.0  15130.0       -1.39        -15.83  18.86   0.04    154.0   
43     -28735.0  28771.0        1.28        -16.46  19.73   0.02    127.0   
50     -44982.0  81314.0        0.62        -16.68  36.19  13.47    142.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
84    -0.94        0.99    57.46    OX40N      NTT    BANKS  
78    -1.00        0.66    34.27    OX40N      NTT       IT  
89    -1.00        0.60    58.23    OX40N      NTT     MISC  
43    -1.00        1.08    26.45    OX40N      NTT   PAINTS  
50    -0.55        1.67    13.05     XY24      NTT   PAINTS

In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
80         TCS  3794.03  4998.00   -17.10       X-LC  37.63  269872.0   
45        INFY  1461.46  2275.00   -10.27       X-LC  52.15  287832.0   
37  HINDUNILVR  2413.81  2723.00   -17.52       X-LC  25.81  258194.0   
58   NESTLEIND  2268.60  2755.00    -9.43       X-LC  49.46  287165.0   
16   BRITANNIA  4983.33  6446.05    16.49       X-LC  66.67  267168.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
80     -26062.0  119985.0        0.55         -8.81  44.46  31.73      1.0   
45      26231.0  119393.0       -0.50         10.03  41.48  55.67      2.0   
37     -12153.0   46785.0       -0.07         -4.50  18.12  12.81      6.0   
58      21739.0   35178.0       -0.17          8.19  12.25  21.44      9.0   
16      37935.0   29362.0        0.23         16.55  10.99  29.35     10.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
80    -0.22        2.00     8.02      X40      BTT       IT  
45     0.22        2.14    16.74      X40      BTT       IT  
37    -0.26        1.92     7.98     XY25      NTT     FMCG  
58     0.62        2.13    14.89     XY25      NTT     FMCG  
16     1.29        1.98    26.95     XY25      ATH     FMCG

In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost     FTT  Dev%_PE Conviction    RSP   Current  \
85   UNIONBANK   123.87   163.0    -2.93       M-LC  61.29  169481.0   
3   ADANIPORTS  1103.69  1583.0     2.03       M-LC  44.09  102595.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%  FTT%   OTT%  CumlRnk  \
85      28641.0  15846.0        0.63         20.34  9.35  31.59     89.0   
3       24233.0   9798.0        0.33         30.92  9.55  43.43     77.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
85     1.81        1.26    47.22     XY24      NTT    BANKS  
3      2.47        0.76    37.89     XY24      NTT     MISC

In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
65        SAIL   130.64   228.00    89.46       M-MC  48.39  227666.0   
51    KPIGREEN   497.21   732.16    25.90       H-SC  62.37  128810.0   
19        CAMS  3643.00  5250.99     7.21       H-SC  81.72  120473.0   
48      JIOFIN   310.58   387.00     7.11       M-LC  93.55  229600.0   
3   ADANIPORTS  1103.69  1583.00     2.03       M-LC  44.09  102595.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
65       2704.0  164944.0        0.39          1.20  72.45  74.53    185.0   
51       3513.0   55697.0        3.24          2.80  43.24  47.25    139.0   
19      18469.0   26552.0        0.23         18.11  22.04  44.14    124.0   
48      12194.0   41305.0        1.41          5.61  17.99  24.61     95.0   
3       24233.0    9798.0        0.33         30.92   9.55  43.43     77.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
65     0.02        1.69    32.18     XY24      BTT    STEEL  
51     0.06        0.96    61.15       MH      ATH    POWER  
19     0.70        0.89    37.29     X40N      ATH     MISC  
48     0.30        1.70    63.25     XY24      BTT  FINANCE  
3      2.47        0.76    37.89     XY24      NTT     MISC

In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
91   WHIRLPOOL  1167.49  2270.00   -38.05       M-SC  90.32  204286.0   
92       WIPRO   243.46   420.00    -4.22       M-LC  71.51  164269.0   
42   INDIAMART  2327.09  4871.06   -47.89       H-SC  88.17  135664.0   
51    KPIGREEN   497.21   732.16    25.90       H-SC  62.37  128810.0   
57  NATIONALUM   189.63   247.44   -49.40       H-MC  76.34  102099.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
91      32665.0  129395.0       -0.93         19.03  63.34   94.43    198.0   
92      13324.0   96130.0       -0.04          8.83  58.52   72.51     69.0   
42      12328.0  122505.0       -1.86         10.00  90.30  109.32    122.0   
51       3513.0   55697.0        3.24          2.80  43.24   47.25    139.0   
57       1595.0   29047.0        0.95          1.59  28.45   30.49     97.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
91     0.25        1.52    50.78       XR      NTT  DURABLES  
92     0.14        1.22    13.15       XR      NTT        IT  
42     0.10        1.01    34.03       AR      ATH      MISC  
51     0.06        0.96    61.15       MH      ATH     POWER  
57     0.05        0.76    34.51       MH      ATH    METALS

In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
23   COFFEEDAY    59.14    80.00   -51.71       L-SC   97.85   72038.0   
55      MASFIN   326.60   402.39   -18.70       H-SC   63.44   92640.0   
4   ALKYLAMINE  2347.98  4546.37     8.10       X-SC  100.00  100564.0   
3   ADANIPORTS  1103.69  1583.00     2.03       M-LC   44.09  102595.0   
87  VALIANTORG   512.64   838.00  -358.89       H-SC    4.30  109980.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
23     -41511.0   81561.0        2.01        -36.56  113.22  35.27    261.0   
55      -5340.0   28079.0       -0.82         -5.45   30.31  23.21    137.0   
4        -399.0   94932.0        3.40         -0.40   94.40  93.63     64.0   
3       24233.0    9798.0        0.33         30.92    9.55  43.43     77.0   
87     -23306.0  107901.0        0.39        -17.49   98.11  63.47    148.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
23    -0.51        0.53    76.32       XR      NTT     HOTELS  
55    -0.19        0.69    34.36       XR      ATH    FINANCE  
4     -0.00        0.75    49.79       SR      ATH  CHEMICALS  
3      2.47        0.76    37.89     XY24      NTT       MISC  
87    -0.22        0.82    84.35       XR      NTT  CHEMICALS

In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction     RSP   Current  \
4   ALKYLAMINE  2347.98  4546.37     8.10       X-SC  100.00  100564.0   
49     JPPOWER    18.73    26.20   -31.87       L-SC   98.92  137071.0   
23   COFFEEDAY    59.14    80.00   -51.71       L-SC   97.85   72038.0   
69         SFL  1016.35  1287.00    43.20       H-SC   96.77  189166.0   
66  SAMMAANCAP   170.35   326.00  -175.00       M-SC   95.70   86190.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
4        -399.0   94932.0        3.40         -0.40   94.40  93.63     64.0   
49      -4771.0   61339.0        0.72         -3.36   44.75  39.88    259.0   
23     -41511.0   81561.0        2.01        -36.56  113.22  35.27    261.0   
69     -73052.0  142877.0       -0.50        -27.86   75.53  26.63    144.0   
66     -16020.0  109410.0        2.10        -15.67  126.94  91.37    206.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
4     -0.00        0.75    49.79       SR      ATH  CHEMICALS  
49    -0.08        1.02    38.91     XY24      NTT      POWER  
23    -0.51        0.53    76.32       XR      NTT     HOTELS  
69    -0.51        1.40    17.61     XY24      NTT       MISC  
66    -0.15        0.64    39.91     XY25      NTT    FINANCE

In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.16
1     25       42.41
2     50       71.05

In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.98
LC    34.44
MC    21.60
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.34
X40      12.34
XY25     12.21
X40N     11.82
XR       10.33
OX40N     8.28
AR        7.82
MH        1.72
X5K       1.50
X200      1.41
SR        1.25
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.87
M-SC    16.21
H-LC    12.61
X-LC    12.34
H-MC     8.64
M-LC     8.41
X-MC     7.59
M-MC     5.00
X-SC     4.42
L-SC     2.48
L-LC     1.08
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                15.30         -1.45   35.36
IT                  11.88        -10.78   65.79
BANKS                8.06         -5.13   48.03
FINANCE              8.04        -11.18   51.43
MISC                 6.59        -21.66   69.24
PAINTS               5.69        -13.99   36.46
ELECTRICAL           5.01         -3.96   48.98
HEALTHCARE           4.52         -5.69   36.56
AUTO                 4.39        -14.34   64.62
AC                   3.40         -5.87   41.97
INSURANCE            3.35         -4.48   37.65
DURABLES             3.18         -3.59   41.45
FOOTWEAR             2.34        -30.59   73.74
METALS               2.26         -1.32   31.36
POWER                1.98         -0.47   44.02
CERAMICS             1.80        -26.16   68.66
STEEL                1.69          1.19   72.45
TRAVEL               1.67        -41.57  102.84
CHEMICALS            1.57        -11.26   96.34
ENGINEERING          1.22        -16.37   64.17
JEWELLERY            1.17        -84.52  131.87
MINING               1.08         -3.47   30.32
CEMENT               1.04        -34.57  103.81
ENTERTAINMENT        0.93        -30.76   96.51
PHARMA               0.75          0.66   19.27
TEXTILES             0.58        -19.02   52.96
HOTELS               0.53        -57.62  113.22

In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      2708849.0     25
XR        1071074.0     13
AR        1016527.0      9
X40        655811.0      9
X40N       566873.0     11
XY25       474169.0      8
OX40N      316777.0     11
SR         175733.0      2
MH          84744.0      2
X200        69753.0      1
X5K         66217.0      2

In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        1972133.0     20
M-SC        1761358.0     21
H-LC         568403.0     11
X-LC         511843.0      7
M-MC         495622.0      4
H-MC         472750.0      7
X-MC         429295.0      6
X-SC         323866.0      5
M-LC         310601.0      6
L-SC         263520.0      4
L-MC          53048.0      1
L-LC          44088.0      1

In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      853325.0      6
M-SC       XY24      813999.0      7
H-SC       AR        521250.0      3
M-MC       XY24      386003.0      3
M-SC       XR        332634.0      4
X-LC       X40       239378.0      2
H-SC       X40N      234454.0      4
H-MC       XY24      224757.0      2
H-SC       XR        214176.0      3
M-LC       XY24      202816.0      4
H-LC       X40N      199490.0      4
M-SC       AR        181552.0      2
           OX40N     171320.0      5
H-LC       X40       160805.0      3
X-MC       X40       153808.0      2
L-SC       XR        142642.0      2
X-SC       XR        122825.0      1
X-LC       AR        119786.0      1
           XY25      111325.0      3
M-MC       XR        109619.0      1
M-SC       XY25      109410.0      1
X-MC       XY25      106188.0      1
           XY24       98409.0      1
M-LC       XR         96130.0      1
X-SC       SR         94932.0      1
H-SC       OX40N      93231.0      3
H-MC       XY25       91503.0      1
M-SC       SR         80801.0      1
H-LC       AR         72138.0      1
M-SC       X40        71642.0      1
H-LC       X200       69753.0      1
X-SC       XY24       68201.0      1
H-LC       X5K        66217.0      2
H-MC       AR         62262.0      1
L-SC       XY24       61339.0      1
           AR         59539.0      1
X-MC       X40N       56572.0      1
H-SC       MH         55697.0      1
L-MC       XR         53048.0      1
L-LC       XY25       44088.0      1
X-LC       X40N       41354.0      1
X-SC       OX40N      37908.0      2
H-MC       X40N       35003.0      1
           X40        30178.0      1
           MH         29047.0      1
X-MC       OX40N      14318.0      1
M-LC       XY25       11655.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 62.0 seconds
